In [109]:
## INPUT =  .npy containing of array of probability values and gound truth label
## OUPUT = Comparison of GT and predicted probabilities. 
import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import listdir
import csv
import sys
path_npy='/media/sagan/Drive1/GTEA/dataset/visualization/npy/seq_11/'



In [110]:
def softmax(mat):
    """Compute softmax values for each sets of scores in x."""
    out = []
    for x in mat:
        e_x = np.exp(x - np.max(x))
        e_x = e_x / e_x.sum()
        out.append(e_x)
    
    return np.asarray(out)

In [111]:
def one_hot(label):
    out = np.zeros((label.shape[0], 11))
    for i in range(len(label)):
        out[i][label[i]] = 1
        
    return out

In [112]:
def vizz(mat):
    mat = np.transpose(mat, (1,0))
    color = []
    for i in range(mat.shape[1]):
        color.append(label_colors[0][:11])
    color = np.asarray(color)
    color = np.transpose(color, (1,0,2))

    white = np.ones(color.shape) * 255
    vector = np.subtract(white, color)
    
    col = []
    for i in range(vector.shape[0]):
        row = []
        for j in range(vector.shape[1]):
            row.append(color[i][j] + vector[i][j] * (1-mat[i][j]))
        col.append(row)
        
    out =  np.asarray(col, dtype=np.uint8)   
    out = cv2.resize(out, (573,100), cv2.INTER_MAX)
    
    return out

In [113]:
# Code to read output generated by aciton set
action_set_path = '/media/sagan/Drive1/GTEA/dataset/action_set/results/'
videos = listdir(action_set_path)
class_map = {'x':0, 'bg':0, 'fold':1, 'pour':2, 'put':3, 'scoop':4, 'shake':5, 'spread':6, 'stir':7, 'take':8, 'open': 9, 'close':10}
label_files = listdir(path_npy)
label_colors = cv2.imread('camvid11.png').astype(np.uint8)
for video in videos:
    if 'S' in video and '.swp' not in video:
        file = open(action_set_path+video, 'r')
        lines=file.readlines()
        predicted = lines[5].split()
        pre_action_set=[]
        for i in range(len(predicted)):
            pre_action_set.append(class_map[predicted[i]])
        pre_action_set=np.asarray(pre_action_set)
        print (pre_action_set.shape)
        
        # code for lstm output and GT 
        label = video+'_lstm_prediction.npy'
        data = np.load(path_npy + label)
        gt = data[:,1]
        probs = data[:,0]
        probs = list(probs)
        probs = np.asarray(probs)
        out_sm = softmax(probs)
        out = vizz(out_sm)
        gt_hot = one_hot(gt)
        pre_action_set_hot = one_hot(pre_action_set)
        gt_out = vizz(gt_hot)
        pre_action_set_out = vizz(pre_action_set_hot)
        black_stripe = np.zeros((20, gt_out.shape[1], 3))
        finall = np.vstack((out, black_stripe , gt_out, black_stripe, pre_action_set_out ))
        print (gt_out.shape)
        cv2.imwrite(path_npy + label[:-4] + '.png', finall)  
            


(405,)
(100, 573, 3)
(502,)
(100, 573, 3)
(869,)
(100, 573, 3)
(387,)
(100, 573, 3)
(930,)
(100, 573, 3)
(1466,)
(100, 573, 3)
(768,)
(100, 573, 3)
(581,)
(100, 573, 3)
(625,)
(100, 573, 3)
(1153,)
(100, 573, 3)
(702,)
(100, 573, 3)
(849,)
(100, 573, 3)
(643,)
(100, 573, 3)
(744,)
(100, 573, 3)
(558,)
(100, 573, 3)
(964,)
(100, 573, 3)
(814,)
(100, 573, 3)
(637,)
(100, 573, 3)
(524,)
(100, 573, 3)
(1259,)
(100, 573, 3)
(551,)
(100, 573, 3)
(692,)
(100, 573, 3)
(543,)
(100, 573, 3)
(387,)
(100, 573, 3)
(800,)
(100, 573, 3)
(418,)
(100, 573, 3)
(926,)
(100, 573, 3)
(573,)
(100, 573, 3)


In [105]:
def levenshtein(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]